In [1]:
import sys
import os
# Get path
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(os.path.join(project_root, 'src'))

from data_cleaning import *

In [2]:
main_dc_nilu()


JSON-filen ble lastet vellykket.

Dataset informasjon:

Antall rader i datasettet: 5222

Genererte verdier per kolonne:
  - generated_NO2: 1268 genererte verdier
  - generated_PM10: 524 genererte verdier
  - generated_PM2.5: 532 genererte verdier

Antall outliers fjernet per kolonne:
  - NO2: referenceTime
2010-01-07 00:00:00+01:00     74.091667
2010-01-13 00:00:00+01:00     75.187500
2010-01-14 00:00:00+01:00     77.004167
2010-01-15 00:00:00+01:00    106.983333
2010-02-02 00:00:00+01:00     84.012500
2010-02-03 00:00:00+01:00     78.891667
2010-11-17 00:00:00+01:00     75.575000
2010-11-19 00:00:00+01:00     80.945833
2010-12-23 00:00:00+01:00     74.150000
2011-01-04 00:00:00+01:00     91.450000
2013-01-29 00:00:00+01:00     80.120833
2013-12-09 00:00:00+01:00     76.295833
2014-12-18 00:00:00+01:00     82.083333
dtype: float64 outliers fjernet
  - PM10: referenceTime
2010-01-07 00:00:00+01:00    54.004167
2010-01-13 00:00:00+01:00    53.500000
2010-01-14 00:00:00+01:00    67.50416

In [3]:
default_clean_frost_data(project_root)


Missing values detected:
mean_wind_speed:
- 2019: 1

No outliers detected

Date gaps detected:
- 2012: 199

Generated values:
- mean_air_temperature: 199
- mean_wind_speed: 200
- total_precipitation: 199

Cleaned data saved to 'h:\my-documents\01-projects\00124-prosjekt_oppgave\data\clean\frost.db' in the table 'weather_data'.


# 🧼 Rensing og klargjøring av data

I denne notebooken klargjør vi de innhentede datasettene fra NILU og Frost for videre analyse og visualisering.

Vi utfører:
- Strukturering av data
- Fjerning av feil og outliers
- Behandling av manglende verdier

All renselogikk ligger i egne funksjoner og moduler.

## 🧽 Om rensing og datakvalitet

Etter at vi har hentet inn data fra Frost (MET) og NILU, er neste steg å gjøre dataene klare til analyse. Dette handler både om å forbedre kvaliteten på dataene – og om å vise at vi forstår hvordan ulike datakilder stiller ulike krav til rensing.

Vi valgte to datasett med kontrasterende egenskaper:

- **Frost**: Strukturert, stabilt og nesten «analyseklart»
- **NILU**: Ujevnt, med mange manglende verdier og måleutfordringer

Denne kontrasten gir oss mulighet til å vise både enkel validering og mer avansert rensing og imputasjon.

---

### 🌫️ NILU – utfordrende, men verdifullt

Datasettet fra NILU inneholder daglige målinger av luftkvalitet i Trondheim (PM10, PM2.5, NO₂), men er preget av:

- Mange manglende verdier
- Ekstreme målinger og støy
- Ujevn dekning over tid og mellom komponenter

Samtidig er dette realistiske utfordringer ved miljødata, og gir oss muligheten til å vise god databehandling i praksis.

#### 🔧 Renseprosessen

1. **Konvertering av datoer**  
`dateTime` ble konvertert til `datetime`-format for å gjøre videre tidsseriebehandling mulig (f.eks. resampling og glatting).

2. **Pivotering av datastruktur**  
Målingene ble transformert slik at én rad tilsvarer én dag, og hver luftkomponent fikk sin egen kolonne. Dette forenkler all videre analyse.

3. **Fjerning av kolonne**  
`Benzo(a)pyrene in PM10` ble fjernet fordi den hadde nesten bare manglende verdier, og ikke er relevant for vår problemstilling.

4. **Fjerning av outliers**  
Målinger som lå mer enn 4 standardavvik fra gjennomsnittet ble fjernet. Disse kan være feil eller kortvarige, ekstreme hendelser som ikke representerer typiske forhold.

5. **Reindeksering av datoer**  
Alle datoer mellom første og siste registrering ble inkludert, også de uten målinger. Dette gjør eventuelle datamangler synlige og muliggjør presis imputasjon.

6. **KNN-imputasjon (k = 100)**  
Manglende verdier ble fylt inn med KNN-imputasjon, som bruker lignende dager til å estimere manglende verdier. Mer om KNN-impulasjon og hvorfor vi har valgt dette kan du lese [her](KNN_imputation.ipynb)


7. **Merking av estimerte verdier**  
Vi opprettet egne `generated_*`-kolonner for å vise hvilke verdier som er estimert. Dette gjør datagrunnlaget transparent og analyserbart.

8. **Glidende gjennomsnitt**  
Vi brukte et 3-dagers glidende gjennomsnitt for å jevne ut tilfeldige variasjoner og fremheve trender.  
– *Hvorfor?* Det gir mer lesbare grafer og et tydeligere bilde av utvikling, men uten å skjule kortsiktige endringer.

9. **Negative verdier satt til 0**  
Luftforurensningsverdier kan ikke være negative. Disse feilene ble rettet for å bevare datasettets troverdighet.

10. **Lagring som JSON**  
Renset data ble lagret som `.json`.  
– Lett å lese og bruke videre i både analyse og visualisering.

---

### 🌦️ Frost – strukturert, men fortsatt valideringsbehov

Frost-dataene er langt mer komplette og standardiserte, men ble likevel validert og bearbeidet for å sikre pålitelighet – og for å kunne kombineres med NILU.

#### 🔧 Renseprosessen

1. **Konvertering av datoformat**  
`referenceTime` ble formatert til datoobjekt for å kunne brukes i tidsserier.

2. **Utvalg av relevante variabler**  
Vi fokuserte på:
  - Temperatur (gjennomsnitt per dag)
  - Nedbør (total per dag)
  - Vindhastighet (gjennomsnitt per dag)

  Disse har dokumentert sammenheng med luftkvalitet. Andre mer usikre eller tekniske variabler ble utelatt.

3. **Pivotering og navneendring**  
Datasettet ble omstrukturert til én rad per dag med mer intuitive kolonnenavn.

4. **Verdikontroll med `ValueRangeValidator`**  
Vi definerte gyldige verdier for Trondheim:
  - Temp: -30 til 40 °C  
  - Nedbør: 0 til 250 mm  
  - Vind: 0 til 60 m/s  

  Verdier utenfor ble fjernet – de er trolig feilregistreringer.

5. **Kontroll for dato-hull**  
Med `DateContinuityValidator` sjekket vi at det ikke manglet perioder i tidsserien. Små hull ble registrert, men ikke imputert – siden Frost generelt har god datadekning.

6. **Lagring som SQLite**  
Værdataene ble lagret i en SQLite-database (`frost.db`) – ideelt for videre analyse med SQL og for eventuell samkjøring med større datasett.

---

### 🔍 Forutsetninger for datarensing

Før vi kan starte datarensingen, må vi ha tilgang til rådataene. Disse hentes ved å:

1. Kjøre `data_collection.ipynb` notebooken først
2. Velge tidsperiode og by for datainnsamling
3. La API-kallene hente data fra Frost og NILU
4. Dataene lagres automatisk i:
   - `data/raw/api_frost_weather.json`
   - `data/raw/api_nilu_air_quality.json`

Når disse filene er på plass, kan vi starte renseprosessen.

---

### 🌦️ Rensing av Frost værdata

Først skal vi rense værdataene fra Frost API. Disse dataene er relativt strukturerte og komplette, men vi må fortsatt:

1. Validere at alle felt har gyldige verdier innenfor forventede grenser
2. Sjekke for manglende verdier og datoer
3. Håndtere eventuelle outliers
4. Lagre resultatet i en SQL database for videre analyse

Vi bruker funksjonen `clean_frost_data()` som håndterer hele renseprosessen gjennom en serie validatorer.

In [ ]:
# Get the default paths
from pathlib import Path
import os

# Get paths relative to the notebook directory
project_root = Path(os.getcwd()).parent
raw_frost_json = project_root / 'data' / 'raw' / 'api_frost_weather.json'
clean_frost_db = project_root / 'data' / 'clean' / 'frost.db'

# First check if source and destination paths exist
if not raw_frost_json.exists():
    print(f"\nError: Source file not found: {raw_frost_json}")
    print("\nYou need to run data collection first to create the input file.")
    print("Please run the data_collection.ipynb notebook first to fetch data from Frost API.")
else:
    print(f"Found source file: {raw_frost_json}")
    
    # Create destination directory if it doesn't exist
    clean_frost_db.parent.mkdir(parents=True, exist_ok=True)
    
    print("\nStarting Frost data cleaning process...")
    print(f"Reading from: {raw_frost_json}")
    print(f"Will save to: {clean_frost_db}\n")
    
    # Clean Frost data and save to SQLite database
    clean_frost_data(str(raw_frost_json), str(clean_frost_db))
    
    if clean_frost_db.exists():
        print(f"\nFrost data cleaning completed.")
        print(f"Results saved to: {clean_frost_db}")
    else:
        print(f"\nWarning: The cleaning process did not create the expected output file!")
        print(f"Expected file: {clean_frost_db}")


---

### 🌫️ Rensing av NILU luftkvalitetsdata

Nå skal vi rense luftkvalitetsdataene fra NILU API. Som beskrevet over, er dette datasettet mer utfordrende med:

- Manglende verdier som må håndteres
- Outliers som må identifiseres og fjernes
- Data som må restruktureres for analyse

Vi bruker funksjonen `main_dc_nilu()` som implementerer alle stegene i renseprosessen beskrevet over.

In [ ]:
# Get the default paths
from pathlib import Path
import os

# Get paths relative to the notebook directory
project_root = Path(os.getcwd()).parent
raw_nilu_json = project_root / 'data' / 'raw' / 'api_nilu_air_quality.json'
clean_nilu_json = project_root / 'data' / 'clean' / 'cleaned_data_nilu.json'

# Check if source and destination paths exist for NILU data
if not raw_nilu_json.exists():
    print(f"\nError: Source file not found: {raw_nilu_json}")
    print("\nYou need to run data collection first to create the input file.")
    print("Please run the data_collection.ipynb notebook first to fetch data from NILU API.")
else:
    print(f"Found source file: {raw_nilu_json}")
    
    # Create destination directory if it doesn't exist
    clean_nilu_json.parent.mkdir(parents=True, exist_ok=True)
    
    print("\nStarting NILU data cleaning process...")
    print(f"Reading from: {raw_nilu_json}")
    print(f"Will save to: {clean_nilu_json}\n")
    
    # Clean NILU data and save to JSON file
    main_dc_nilu(str(raw_nilu_json), str(clean_nilu_json))
    
    if clean_nilu_json.exists():
        print(f"\nNILU data cleaning completed.")
        print(f"Results saved to: {clean_nilu_json}")
    else:
        print(f"\nWarning: The cleaning process did not create the expected output file!")
        print(f"Expected file: {clean_nilu_json}")


---

### 🧠 Hvorfor dette er en god tilnærming

Vi har valgt rensestrategier basert på datakildenes egenskaper:

- **NILU** krevde:
  - Komplett rekonstruering av datastrukturen
  - Avansert imputasjon og glatting
  - Transparens rundt hvilke verdier som er estimert

- **Frost** krevde:
  - Kontroll av gyldige verdier og datoer
  - Lett justering og tilrettelegging for videre analyse

Dette viser:
- Forståelse av hva som påvirker datakvalitet
- Evne til å tilpasse metoder etter utfordring
- Fokus på transparens og sporbarhet

Resultatet er et datasett som både er **ryddet og dokumentert**, og klart for videre analyse, visualisering og modellering.

---

## 📈 Neste steg: Analyse og visualisering

Nå som begge datasett er renset og klargjort, er vi klare for å utforske innholdet mer inngående. I neste notebook skal vi:

- Beregne statistiske mål som gjennomsnitt, median og standardavvik
- Undersøke sammenhenger og trender i luftkvalitet og værdata over tid
- Lage visuelle fremstillinger som gjør datamønstre lettere å forstå


### [**Videre til analyse og visualisering**](02_data_analysis_and_visualisation.ipynb)
##### [**Til samlesiden**](../docs/samleside.md)

